## Infer schema directly, self correct and pick optimal query using CodeChat-Bison

#### Imports 

In [1]:
from vertexai.language_models import CodeChatSession
from vertexai.language_models import CodeChatModel
from google.cloud import bigquery
import pandas as pd
import logging 
import time
import os 

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup essentials

In [3]:
# Adjust display settings
pd.set_option('display.max_colwidth', None)  # Set max column width to None to show all content
pd.set_option('display.expand_frame_repr', False)  # Prevent truncation of DataFrame HTML representation


In [4]:
SERVICE_ACCOUNT_CREDENTIALS = './../credentials/vai-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_CREDENTIALS

In [5]:
PROJECT_ID = 'arun-genai-bb'
MODEL_NAME = 'codechat-bison@latest'
TEMPERATURE = 1 
MAX_OUTPUT_TOKENS = 2048  # length of the output response | overridding the default value which is 128
# TOP_P = 0.95  # default value
# TOP_K = 40  # default value
LOCATION = 'us-central1'

In [6]:
DATASET = 'flight_reservations'
TABLES = ['customers', 'flights', 'reservations', 'transactions', 'loyality_points']

In [7]:
model = CodeChatModel.from_pretrained(MODEL_NAME)
bq_client = bigquery.Client()

In [8]:
query = f"""
    SELECT *
    FROM `{PROJECT_ID}.{DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ({','.join([f'"{table}"' for table in TABLES])})
"""
logger.info(query)


    SELECT *
    FROM `arun-genai-bb.flight_reservations.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ("customers","flights","reservations","transactions","loyality_points")



In [9]:
schema_columns = bq_client.query(query=query).to_dataframe()
schema_columns

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,arun-genai-bb,flight_reservations,transactions,transaction_id,transaction_id,INT64,None,NULL,None
1,arun-genai-bb,flight_reservations,transactions,reservation_id,reservation_id,INT64,None,NULL,None
2,arun-genai-bb,flight_reservations,transactions,amount,amount,FLOAT64,None,NULL,None
3,arun-genai-bb,flight_reservations,transactions,transaction_datetime,transaction_datetime,DATETIME,None,NULL,None
4,arun-genai-bb,flight_reservations,reservations,reservation_id,reservation_id,INT64,None,NULL,None
5,arun-genai-bb,flight_reservations,reservations,customer_id,customer_id,INT64,None,NULL,None
6,arun-genai-bb,flight_reservations,reservations,flight_id,flight_id,INT64,None,NULL,None
7,arun-genai-bb,flight_reservations,reservations,reservation_datetime,reservation_datetime,DATETIME,None,NULL,None
8,arun-genai-bb,flight_reservations,reservations,status,status,STRING,None,NULL,None
9,arun-genai-bb,flight_reservations,flights,flight_id,flight_id,INT64,None,NULL,None


In [10]:
schema_columns = schema_columns.to_markdown(index=False)
logger.info(schema_columns)

| table_catalog   | table_schema        | table_name   | column_name          | field_path           | data_type   | description   | collation_name   | rounding_mode   |
|:----------------|:--------------------|:-------------|:---------------------|:---------------------|:------------|:--------------|:-----------------|:----------------|
| arun-genai-bb   | flight_reservations | transactions | transaction_id       | transaction_id       | INT64       |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions | reservation_id       | reservation_id       | INT64       |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions | amount               | amount               | FLOAT64     |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions | transaction_datetime | transaction_datetime | DATETIME    |               | NULL             

Utility function to convert text into SQL and automatically correct the SQL query if any execution errors occur. <br>
The function also collects the successful query executions, ranks them by latency of execution, and returns the fastest. <br>
Additionally, it provides an option to retrieve all the successful queries and their latencies as a dataframe

In [11]:
def generate_and_execute_sql(prompt, max_tries=5, return_all=False):
    """
    Generate an SQL query using the code_gen_model, execute it using bq_client, and rank successful queries by latency.
    
    Args:
    - prompt (str): Prompt to provide to the model for generating SQL.
    - max_tries (int): Maximum number of attempts to generate and execute SQL.
    - return_all (bool): Flag to determine whether to return all successful queries or only the fastest.
    
    Returns:
    - dict: A dictionary containing the fastest dataframe or all successful dataframes, or error messages and prompt evolution.
    """
    
    tries = 0
    error_messages = []
    prompts = [prompt]
    successful_queries = []

    chat_session = CodeChatSession(model=model, 
                                   temperature=TEMPERATURE, 
                                   max_output_tokens=MAX_OUTPUT_TOKENS)
    
    while tries < max_tries:
        logger.info(f'TRIAL: {tries+1}')
        try:
            # Predict SQL using the model
            start_time = time.time()
            response = chat_session.send_message(prompt, temperature=TEMPERATURE, max_output_tokens=MAX_OUTPUT_TOKENS)
            generated_sql_query = response.text
            generated_sql_query = '\n'.join(generated_sql_query.split('\n')[1:-1])
            logger.info('-' * 50)
            logger.info(generated_sql_query)
            logger.info('-' * 50)
            # Execute SQL using BigQuery client
            df = bq_client.query(generated_sql_query).to_dataframe()
            latency = time.time() - start_time
            successful_queries.append({
                "query": generated_sql_query,
                "dataframe": df,
                "latency": latency
            })
            logger.info('SUCCEEDED')
            # Evolve the prompt for success path to optimize the last successful query for latency
            if len(successful_queries) > 1:
                prompt = f"""Modify the last successful SQL query by making changes to it and optimizing it for latency. 
            ENSURE that the NEW QUERY is DIFFERENT from the previous one while prioritizing faster execution. 
            The last successful query was:
            {successful_queries[-1]["query"]}"""
        except Exception as e:
            logger.error('FAILED')
            # Catch the error, store the message, and try again
            msg = str(e)
            error_messages.append(msg)
            # Evolve the prompt by appending the error message and asking the model to correct it
            prompt = f"""Encountered an error: {msg}. 
To address this, please generate an alternative SQL query response that avoids this specific error. 
Follow the instructions mentioned above to remediate the error. 

Modify the below SQL query to resolve the issue and ensure it is not a repetition of all previously generated queries.
{generated_sql_query}
 
Ensure the revised SQL query aligns precisely with the requirements outlined in the initial question.
Additionally, please optimize the query for latency while maintaining correctness and efficiency."""
            prompts.append(prompt)
        logger.info('=' * 100)
        tries += 1
    # If no successful queries
    if len(successful_queries) == 0:
        return {
            "error": "All attempts exhausted.",
            "prompts": prompts,
            "errors": error_messages
        }
    
    # Sort successful queries by latency
    successful_queries.sort(key=lambda x: x['latency'])
    
    if return_all:
        df = pd.DataFrame([(q["query"], q["dataframe"], q["latency"]) for q in successful_queries], columns=["Query", "Result", "Latency"])
        return {
            "dataframe": df
        }
    else:
        return {
            "fastest_query": successful_queries[0]["query"],
            "result": successful_queries[0]["dataframe"],
            "latency": successful_queries[0]["latency"]
        }

### Test text to SQL scenarios

Construct the SEED prompt

In [12]:
seed_prompt = """
Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.\n
QUESTION:
{}\n
SCHEMA:
{}\n
IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
--
Ensure your SQL query accurately defines both the start and end of the DATETIME range.
"""
logger.info(seed_prompt)


Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.

QUESTION:
{}

SCHEMA:
{}

IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
--
Ensure your SQL query accurately defines both the start and end of the DATETIME range.



#### Scenario 1: Retrieve Active Reservations for a Specific Date Range

For this scenario, you want to find all active reservations within a specific date range.

In [13]:
question = "Provide a list of all flight reservations from October 10th to October 15th, 2023"

In [14]:
prompt = seed_prompt.format(question, schema_columns)
logger.info(prompt)


Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.

QUESTION:
Provide a list of all flight reservations from October 10th to October 15th, 2023

SCHEMA:
| table_catalog   | table_schema        | table_name   | column_name          | field_path           | data_type   | description   | collation_name   | rounding_mode   |
|:----------------|:--------------------|:-------------|:---------------------|:---------------------|:------------|:--------------|:-----------------|:----------------|
| arun-genai-bb   | flight_reservations | transactions | transaction_id       | transaction_id       | INT64       |               | NULL             |                 |
| arun-genai-bb 

In [15]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)
sql_output = response['dataframe']
sql_output


TRIAL: 1
--------------------------------------------------
SELECT
  r.reservation_id,
  r.customer_id,
  r.flight_id,
  r.reservation_datetime,
  r.status,
  f.origin,
  f.destination,
  f.departure_datetime,
  f.arrival_datetime,
  f.carrier,
  f.price
FROM arun-genai-bb.flight_reservations.reservations r
JOIN arun-genai-bb.flight_reservations.flights f
ON CAST(r.flight_id AS STRING) = CAST(f.flight_id AS STRING)
WHERE r.reservation_datetime BETWEEN '2023-10-10 00:00:00' AND '2023-10-15 23:59:59';
--------------------------------------------------
SUCCEEDED
TRIAL: 2
--------------------------------------------------
SELECT
  r.reservation_id,
  r.customer_id,
  r.flight_id,
  r.reservation_datetime,
  r.status,
  f.origin,
  f.destination,
  f.departure_datetime,
  f.arrival_datetime,
  f.carrier,
  f.price
FROM arun-genai-bb.flight_reservations.reservations r
JOIN arun-genai-bb.flight_reservations.flights f
ON r.flight_id = f.flight_id
WHERE r.reservation_datetime BETWEEN TIMESTAMP(

CPU times: user 312 ms, sys: 65.8 ms, total: 378 ms
Wall time: 1min 29s


,Query,Result,Latency
0,"SELECT\n r.reservation_id,\n r.customer_id,\n r.flight_id,\n r.reservation_datetime,\n r.status,\n f.origin,\n f.destination,\n f.departure_datetime,\n f.arrival_datetime,\n f.carrier,\n f.price\nFROM arun-genai-bb.flight_reservations.reservations r\nJOIN arun-genai-bb.flight_reservations.flights f\nON CAST(r.flight_id AS STRING) = CAST(f.flight_id AS STRING)\nWHERE r.reservation_datetime BETWEEN '2023-10-10 00:00:00' AND '2023-10-15 23:59:59';",reservation_id customer_id flight_id reservation_datetime status origin destination departure_datetime arrival_datetime carrier price 0 6 6 6 2023-10-10 10:00:00 Confirmed SEA JFK 2023-11-25 06:00:00 2023-11-25 14:30:00 United 550.0 1 7 6 7 2023-10-12 11:30:00 Confirmed JFK MIA 2023-11-27 20:00:00 2023-11-27 23:30:00 American 380.0 2 8 8 8 2023-10-15 13:20:00 Confirmed MIA JFK 2023-11-30 10:00:00 2023-11-30 13:30:00 American 380.0,16.119379
1,"SELECT\n r.reservation_id,\n r.customer_id,\n r.flight_id,\n r.reservation_datetime,\n r.status,\n f.origin,\n f.destination,\n f.departure_datetime,\n f.arrival_datetime,\n f.carrier,\n f.price\nFROM arun-genai-bb.flight_reservations.reservations r\nJOIN arun-genai-bb.flight_reservations.flights f\nON CAST(r.flight_id AS STRING) = CAST(f.flight_id AS STRING)\nWHERE r.reservation_datetime BETWEEN '2023-10-10 00:00:00' AND '2023-10-15 23:59:59';",reservation_id customer_id flight_id reservation_datetime status origin destination departure_datetime arrival_datetime carrier price 0 6 6 6 2023-10-10 10:00:00 Confirmed SEA JFK 2023-11-25 06:00:00 2023-11-25 14:30:00 United 550.0 1 7 6 7 2023-10-12 11:30:00 Confirmed JFK MIA 2023-11-27 20:00:00 2023-11-27 23:30:00 American 380.0 2 8 8 8 2023-10-15 13:20:00 Confirmed MIA JFK 2023-11-30 10:00:00 2023-11-30 13:30:00 American 380.0,19.265791
2,"SELECT\n r.reservation_id,\n r.customer_id,\n r.flight_id,\n r.reservation_datetime,\n r.status,\n f.origin,\n f.destination,\n f.departure_datetime,\n f.arrival_datetime,\n f.carrier,\n f.price\nFROM arun-genai-bb.flight_reservations.reservations r\nJOIN arun-genai-bb.flight_reservations.flights f\nON r.flight_id = f.flight_id\nWHERE r.reservation_datetime BETWEEN '2023-10-10 00:00:00' AND '2023-10-15 23:59:59'\nORDER BY r.reservation_datetime ASC;",reservation_id customer_id flight_id reservation_datetime status origin destination departure_datetime arrival_datetime carrier price 0 6 6 6 2023-10-10 10:00:00 Confirmed SEA JFK 2023-11-25 06:00:00 2023-11-25 14:30:00 United 550.0 1 7 6 7 2023-10-12 11:30:00 Confirmed JFK MIA 2023-11-27 20:00:00 2023-11-27 23:30:00 American 380.0 2 8 8 8 2023-10-15 13:20:00 Confirmed MIA JFK 2023-11-30 10:00:00 2023-11-30 13:30:00 American 380.0,21.717461
3,"SELECT\n r.reservation_id,\n r.customer_id,\n r.flight_id,\n r.reservation_datetime,\n r.status,\n f.origin,\n f.destination,\n f.departure_datetime,\n f.arrival_datetime,\n f.carrier,\n f.price\nFROM arun-genai-bb.flight_reservations.reservations r\nJOIN arun-genai-bb.flight_reservations.flights f\nON r.flight_id = f.flight_id\nWHERE r.reservation_datetime BETWEEN '2023-10-10 00:00:00' AND '2023-10-15 23:59:59'\nORDER BY r.reservation_datetime ASC\nLIMIT 1000;",reservation_id customer_id flight_id reservation_datetime status origin destination departure_datetime arrival_datetime carrier price 0 6 6 6 2023-10-10 10:00:00 Confirmed SEA JFK 2023-11-25 06:00:00 2023-11-25 14:30:00 United 550.0 1 7 6 7 2023-10-12 11:30:00 Confirmed JFK MIA 2023-11-27 20:00:00 2023-11-27 23:30:00 American 380.0 2 8 8 8 2023-10-15 13:20:00 Confirmed MIA JFK 2023-11-30 10:00:00 2023-11-30 13:30:00 American 380.0,22.154007


In [16]:
result_df = sql_output.loc[0, 'Result']
result_df


,reservation_id,customer_id,flight_id,reservation_datetime,status,origin,destination,departure_datetime,arrival_datetime,carrier,price
0,6,6,6,2023-10-10 10:00:00,Confirmed,SEA,JFK,2023-11-25 06:00:00,2023-11-25 14:30:00,United,550.0
1,7,6,7,2023-10-12 11:30:00,Confirmed,JFK,MIA,2023-11-27 20:00:00,2023-11-27 23:30:00,American,380.0
2,8,8,8,2023-10-15 13:20:00,Confirmed,MIA,JFK,2023-11-30 10:00:00,2023-11-30 13:30:00,American,380.0


#### Scenario 2: Identify customers who made reservations in the past N days.

In [17]:
question = "Identify all customers who have made flight reservations within the last 7 days."

In [18]:
prompt = seed_prompt.format(question, schema_columns)

In [19]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)
sql_output = response['dataframe']
sql_output

TRIAL: 1
--------------------------------------------------
SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  c.email,
  t.transaction_datetime AS reservation_datetime
FROM arun-genai-bb.flight_reservations.transactions t
JOIN arun-genai-bb.flight_reservations.reservations r
ON t.reservation_id = r.reservation_id
JOIN arun-genai-bb.flight_reservations.customers c
ON r.customer_id = c.customer_id
WHERE t.transaction_datetime >= DATE_SUB(CURRENT_DATETIME(), INTERVAL 7 DAY)
AND t.transaction_datetime < CURRENT_DATETIME()
--------------------------------------------------
SUCCEEDED
TRIAL: 2
--------------------------------------------------
SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  c.email,
  t.transaction_datetime AS reservation_datetime
FROM arun-genai-bb.flight_reservations.transactions t
JOIN arun-genai-bb.flight_reservations.reservations r
ON t.reservation_id = r.reservation_id
JOIN arun-genai-bb.flight_reservations.customers c
ON r.customer_id = c.customer_id
WH

CPU times: user 224 ms, sys: 59.8 ms, total: 284 ms
Wall time: 1min 20s


,Query,Result,Latency
0,"SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n c.email,\n t.transaction_datetime AS reservation_datetime\nFROM arun-genai-bb.flight_reservations.transactions t\nJOIN arun-genai-bb.flight_reservations.reservations r\nON t.reservation_id = r.reservation_id\nJOIN arun-genai-bb.flight_reservations.customers c\nON r.customer_id = c.customer_id\nWHERE t.transaction_datetime BETWEEN DATE_SUB(CURRENT_DATETIME(), INTERVAL 7 DAY) AND DATE_ADD(CURRENT_DATETIME(), INTERVAL 1 DAY)",customer_id first_name last_name email reservation_datetime 0 11 Ian Somerhalder ian.s@example.com 2023-10-28 17:11:00 1 13 Kate Winslet kate.w@example.com 2023-11-02 08:21:00 2 15 Mary Jane mary.j@example.com 2023-11-04 10:46:00,15.924800
1,"SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n c.email,\n t.transaction_datetime AS reservation_datetime\nFROM arun-genai-bb.flight_reservations.transactions t\nJOIN arun-genai-bb.flight_reservations.reservations r\nON t.reservation_id = r.reservation_id\nJOIN arun-genai-bb.flight_reservations.customers c\nON r.customer_id = c.customer_id\nWHERE t.transaction_datetime >= DATE_SUB(CURRENT_DATETIME(), INTERVAL 7 DAY)\nAND t.transaction_datetime < CURRENT_DATETIME()",customer_id first_name last_name email reservation_datetime 0 11 Ian Somerhalder ian.s@example.com 2023-10-28 17:11:00 1 13 Kate Winslet kate.w@example.com 2023-11-02 08:21:00,18.279402
2,"SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n c.email,\n t.transaction_datetime AS reservation_datetime\nFROM arun-genai-bb.flight_reservations.transactions t\nJOIN arun-genai-bb.flight_reservations.reservations r\nON t.reservation_id = r.reservation_id\nJOIN arun-genai-bb.flight_reservations.customers c\nON r.customer_id = c.customer_id\nWHERE t.transaction_datetime >= DATE_SUB(CURRENT_DATETIME(), INTERVAL 7 DAY)\nAND t.transaction_datetime < DATE_ADD(CURRENT_DATETIME(), INTERVAL 1 DAY)",customer_id first_name last_name email reservation_datetime 0 11 Ian Somerhalder ian.s@example.com 2023-10-28 17:11:00 1 13 Kate Winslet kate.w@example.com 2023-11-02 08:21:00 2 15 Mary Jane mary.j@example.com 2023-11-04 10:46:00,22.835325


In [20]:
result_df = sql_output.loc[0, 'Result']
result_df

,customer_id,first_name,last_name,email,reservation_datetime
0,11,Ian,Somerhalder,ian.s@example.com,2023-10-28 17:11:00
1,13,Kate,Winslet,kate.w@example.com,2023-11-02 08:21:00
2,15,Mary,Jane,mary.j@example.com,2023-11-04 10:46:00


### Scenario 3: Calculate Monthly Revenue
Calculate the total revenue generated from transactions for a given month and year.

In [21]:
question = "Calculate the total revenue generated from transactions in October 2023, specifically from all reservations with a Confirmed status."

In [22]:
prompt = seed_prompt.format(question, schema_columns)

In [23]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)
sql_output = response['dataframe']
sql_output

TRIAL: 1
--------------------------------------------------
SELECT
  SUM(transactions.amount) AS total_revenue_usd
FROM
  arun-genai-bb.flight_reservations.transactions AS transactions
JOIN
  arun-genai-bb.flight_reservations.reservations AS reservations
ON
  transactions.reservation_id = reservations.reservation_id
WHERE
  reservations.status = 'Confirmed'
  AND transactions.transaction_datetime BETWEEN '2023-10-01 00:00:00' AND '2023-10-31 23:59:59';
--------------------------------------------------
SUCCEEDED
TRIAL: 2
--------------------------------------------------
SELECT
  SUM(transactions.amount) AS total_revenue_usd
FROM
  arun-genai-bb.flight_reservations.transactions AS transactions
JOIN
  arun-genai-bb.flight_reservations.reservations AS reservations
ON
  transactions.reservation_id = CAST(reservations.reservation_id AS INT64)
WHERE
  reservations.status = 'Confirmed'
  AND transactions.transaction_datetime BETWEEN '2023-10-01 00:00:00' AND '2023-10-31 23:59:59';
----------

CPU times: user 289 ms, sys: 82.4 ms, total: 371 ms
Wall time: 1min 32s


,Query,Result,Latency
0,SELECT\n SUM(transactions.amount) AS total_revenue_usd\nFROM\n arun-genai-bb.flight_reservations.transactions AS transactions\nJOIN\n arun-genai-bb.flight_reservations.reservations AS reservations\nON\n transactions.reservation_id = reservations.reservation_id\nWHERE\n reservations.status = 'Confirmed'\n AND transactions.transaction_datetime >= '2023-10-01 00:00:00'\n AND transactions.transaction_datetime < '2023-11-01 00:00:00';,total_revenue_usd 0 3860.0,15.278415
1,SELECT\n SUM(transactions.amount) AS total_revenue_usd\nFROM\n arun-genai-bb.flight_reservations.transactions AS transactions\nJOIN\n arun-genai-bb.flight_reservations.reservations AS reservations\nON\n transactions.reservation_id = reservations.reservation_id\nWHERE\n reservations.status = 'Confirmed'\n AND transactions.transaction_datetime BETWEEN '2023-10-01 00:00:00' AND '2023-10-31 23:59:59';,total_revenue_usd 0 3860.0,19.683552
2,SELECT\n SUM(transactions.amount) AS total_revenue_usd\nFROM\n arun-genai-bb.flight_reservations.transactions AS transactions\nJOIN\n arun-genai-bb.flight_reservations.reservations AS reservations\nON\n transactions.reservation_id = CAST(reservations.reservation_id AS INT64)\nWHERE\n reservations.status = 'Confirmed'\n AND transactions.transaction_datetime BETWEEN '2023-10-01 00:00:00' AND '2023-10-31 23:59:59';,total_revenue_usd 0 3860.0,19.699193
3,SELECT\n SUM(transactions.amount) AS total_revenue_usd\nFROM\n arun-genai-bb.flight_reservations.transactions AS transactions\nJOIN\n arun-genai-bb.flight_reservations.reservations AS reservations\nON\n transactions.reservation_id = reservations.reservation_id\nWHERE\n reservations.status = 'Confirmed'\n AND transactions.transaction_datetime BETWEEN '2023-10-01 00:00:00' AND '2023-10-31 23:59:59';,total_revenue_usd 0 3860.0,24.675394


In [24]:
result_df = sql_output.loc[0, 'Result']
result_df

,total_revenue_usd
0,3860.0


### Scenario 4: Popular Flight Times
Identify the most popular departure hours or days for a given day or month or year.

In [25]:
question = "Determine the departure months with the highest frequency for the year 2023."

In [26]:
prompt = seed_prompt.format(question, schema_columns)

In [27]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)
sql_output = response['dataframe']
sql_output

TRIAL: 1
--------------------------------------------------
SELECT DATE_TRUNC(transaction_datetime, MONTH) AS departure_month,
       COUNT(DISTINCT transaction_id) AS num_transactions
FROM arun-genai-bb.flight_reservations.transactions
WHERE DATE_TRUNC(transaction_datetime, YEAR) = '2023'
GROUP BY departure_month
ORDER BY num_transactions DESC
LIMIT 10;
--------------------------------------------------
FAILED
TRIAL: 2
--------------------------------------------------
SELECT DATE_TRUNC(transaction_datetime, MONTH) AS departure_month,
       COUNT(DISTINCT transaction_id) AS num_transactions
FROM arun-genai-bb.flight_reservations.transactions
WHERE YEAR(transaction_datetime) = 2023
GROUP BY departure_month
ORDER BY num_transactions DESC
LIMIT 10;
--------------------------------------------------
FAILED
TRIAL: 3
--------------------------------------------------
SELECT DATE_TRUNC(transaction_datetime, MONTH) AS departure_month,
       COUNT(DISTINCT transaction_id) AS num_transactions

CPU times: user 229 ms, sys: 65.2 ms, total: 294 ms
Wall time: 1min 8s


,Query,Result,Latency
0,"SELECT DATE_TRUNC(transaction_datetime, MONTH) AS departure_month,\n COUNT(DISTINCT transaction_id) AS num_transactions\nFROM arun-genai-bb.flight_reservations.transactions\nWHERE EXTRACT(YEAR FROM transaction_datetime) = 2023\nGROUP BY departure_month\nORDER BY num_transactions DESC\nLIMIT 10;",departure_month num_transactions 0 2023-10-01 10 1 2023-11-01 6,14.605921
1,"SELECT DATE_TRUNC(transaction_datetime, MONTH) AS departure_month,\n COUNT(DISTINCT transaction_id) AS num_transactions\nFROM (\n SELECT transaction_datetime, transaction_id\n FROM arun-genai-bb.flight_reservations.transactions\n WHERE transaction_datetime BETWEEN '2023-01-01' AND '2023-12-31'\n) AS t\nGROUP BY departure_month\nORDER BY num_transactions DESC\nLIMIT 10;",departure_month num_transactions 0 2023-10-01 10 1 2023-11-01 6,15.977074
2,"SELECT DATE_TRUNC(transaction_datetime, MONTH) AS departure_month,\n COUNT(DISTINCT transaction_id) AS num_transactions\nFROM arun-genai-bb.flight_reservations.transactions\nWHERE transaction_datetime BETWEEN '2023-01-01' AND '2023-12-31'\nGROUP BY departure_month\nORDER BY num_transactions DESC\nLIMIT 10;",departure_month num_transactions 0 2023-10-01 10 1 2023-11-01 6,20.122611


In [28]:
result_df = sql_output.loc[0, 'Result']
result_df

,departure_month,num_transactions
0,2023-10-01,10
1,2023-11-01,6


### Scenario 5: Customer Age Group
Group customers by age brackets and count the number in each bracket.

In [29]:
question = "Group customers into five distinct age brackets and count the number of customers in each bracket."

In [30]:
prompt = seed_prompt.format(question, schema_columns)

In [31]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)
sql_output = response['dataframe']
sql_output

TRIAL: 1
--------------------------------------------------
WITH customer_age AS (
  SELECT
    customer_id,
    DATE_DIFF(CURRENT_DATE(), date_of_birth, YEAR) AS age
  FROM
    `arun-genai-bb.flight_reservations.customers`
)

SELECT
  CASE
    WHEN age < 20 THEN '0-19'
    WHEN age < 30 THEN '20-29'
    WHEN age < 40 THEN '30-39'
    WHEN age < 50 THEN '40-49'
    ELSE '50+'
  END AS age_bracket,
  COUNT(customer_id) AS num_customers
FROM
  customer_age
GROUP BY
  1
--------------------------------------------------
SUCCEEDED
TRIAL: 2
--------------------------------------------------
WITH customer_age AS (
  SELECT
    customer_id,
    DATE_DIFF(CURRENT_DATE(), date_of_birth, YEAR) AS age
  FROM
    `arun-genai-bb.flight_reservations.customers`
)

SELECT
  CASE
    WHEN age < 20 THEN '0-19'
    WHEN age < 30 THEN '20-29'
    WHEN age < 40 THEN '30-39'
    WHEN age < 50 THEN '40-49'
    ELSE '50+'
  END AS age_bracket,
  COUNT(customer_id) AS num_customers
FROM
  customer_age
GROUP BY

CPU times: user 311 ms, sys: 73.1 ms, total: 385 ms
Wall time: 1min 21s


,Query,Result,Latency
0,"WITH customer_age AS (\n SELECT\n customer_id,\n EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM date_of_birth) AS age\n FROM\n `arun-genai-bb.flight_reservations.customers`\n)\n\nSELECT\n age_bracket,\n COUNT(customer_id) AS num_customers\nFROM (\n SELECT\n customer_id,\n CASE\n WHEN age < 20 THEN '0-19'\n WHEN age < 30 THEN '20-29'\n WHEN age < 40 THEN '30-39'\n WHEN age < 50 THEN '40-49'\n ELSE '50+'\n END AS age_bracket\n FROM\n customer_age\n) AS age_brackets\nGROUP BY\n 1",age_bracket num_customers 0 30-39 6 1 40-49 5 2 20-29 3 3 50+ 6,14.066733
1,"WITH customer_age AS (\n SELECT\n customer_id,\n EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM date_of_birth) AS age\n FROM\n `arun-genai-bb.flight_reservations.customers`\n)\n\nSELECT\n age_bracket,\n COUNT(customer_id) AS num_customers\nFROM (\n SELECT\n customer_id,\n CASE\n WHEN age < 20 THEN '0-19'\n WHEN age < 30 THEN '20-29'\n WHEN age < 40 THEN '30-39'\n WHEN age < 50 THEN '40-49'\n ELSE '50+'\n END AS age_bracket\n FROM\n customer_age\n) AS age_brackets\nGROUP BY\n 1",age_bracket num_customers 0 30-39 6 1 40-49 5 2 20-29 3 3 50+ 6,14.597461
2,"WITH customer_age AS (\n SELECT\n customer_id,\n DATE_DIFF(CURRENT_DATE(), date_of_birth, YEAR) AS age\n FROM\n `arun-genai-bb.flight_reservations.customers`\n)\n\nSELECT\n CASE\n WHEN age < 20 THEN '0-19'\n WHEN age < 30 THEN '20-29'\n WHEN age < 40 THEN '30-39'\n WHEN age < 50 THEN '40-49'\n ELSE '50+'\n END AS age_bracket,\n COUNT(customer_id) AS num_customers\nFROM\n customer_age\nGROUP BY\n 1",age_bracket num_customers 0 30-39 6 1 40-49 5 2 20-29 3 3 50+ 6,15.084443
3,"WITH customer_age AS (\n SELECT\n customer_id,\n EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM date_of_birth) AS age\n FROM\n `arun-genai-bb.flight_reservations.customers`\n)\n\nSELECT\n CASE\n WHEN age < 20 THEN '0-19'\n WHEN age < 30 THEN '20-29'\n WHEN age < 40 THEN '30-39'\n WHEN age < 50 THEN '40-49'\n ELSE '50+'\n END AS age_bracket,\n COUNT(customer_id) AS num_customers\nFROM\n customer_age\nGROUP BY\n 1",age_bracket num_customers 0 30-39 6 1 40-49 5 2 20-29 3 3 50+ 6,15.687930
4,"WITH customer_age AS (\n SELECT\n customer_id,\n DATE_DIFF(CURRENT_DATE(), date_of_birth, YEAR) AS age\n FROM\n `arun-genai-bb.flight_reservations.customers`\n)\n\nSELECT\n CASE\n WHEN age < 20 THEN '0-19'\n WHEN age < 30 THEN '20-29'\n WHEN age < 40 THEN '30-39'\n WHEN age < 50 THEN '40-49'\n ELSE '50+'\n END AS age_bracket,\n COUNT(customer_id) AS num_customers\nFROM\n customer_age\nGROUP BY\n 1",age_bracket num_customers 0 30-39 6 1 40-49 5 2 20-29 3 3 50+ 6,22.166036


In [32]:
result_df = sql_output.loc[0, 'Result']
result_df

,age_bracket,num_customers
0,30-39,6
1,40-49,5
2,20-29,3
3,50+,6


### Scenario 6: Age Calculation
Calculate the age of customers based on their date of birth and filter those who are above X years old.

In [33]:
question = "Identify and rank all customers aged 18+ who have `Confirmed` reservations for the current month, ordered by their age. Make sure to display their ages in the result."

In [34]:
prompt = seed_prompt.format(question, schema_columns)

In [35]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)
sql_output = response['dataframe']
sql_output

TRIAL: 1
--------------------------------------------------
SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  DATEDIFF(CURRENT_DATE(), c.date_of_birth) AS age,
  r.reservation_id,
  r.status
FROM flight_reservations.customers c
JOIN flight_reservations.reservations r
ON c.customer_id = r.customer_id
WHERE r.status = 'Confirmed'
AND r.reservation_datetime >= DATE_TRUNC(CURRENT_DATE(), MONTH)
AND r.reservation_datetime < DATE_ADD(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)
AND c.date_of_birth < DATE_SUB(CURRENT_DATE(), INTERVAL 18 YEAR)
ORDER BY age DESC;
--------------------------------------------------
FAILED
TRIAL: 2
--------------------------------------------------
SELECT
  c.customer_id,
  c.first_name,
  c.last_name,
  DATE_DIFF(CURRENT_DATE(), c.date_of_birth, YEAR) AS age,
  r.reservation_id,
  r.status
FROM flight_reservations.customers c
JOIN flight_reservations.reservations r
ON c.customer_id = r.customer_id
WHERE r.status = 'Confirmed'
AND r.reservation_date

CPU times: user 304 ms, sys: 86.2 ms, total: 390 ms
Wall time: 1min 20s


,Query,Result,Latency
0,"SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM c.date_of_birth) AS age,\n r.reservation_id,\n r.status\nFROM flight_reservations.customers c\nJOIN flight_reservations.reservations r\nON c.customer_id = r.customer_id\nWHERE r.status = 'Confirmed'\nAND r.reservation_datetime >= DATE_TRUNC(CURRENT_DATE(), MONTH)\nAND r.reservation_datetime < DATE_ADD(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)\nAND c.date_of_birth < DATE_SUB(CURRENT_DATE(), INTERVAL 18 YEAR)\nORDER BY age DESC;",customer_id first_name last_name age reservation_id status 0 17 Olivia Newton 75 17 Confirmed 1 16 Nick Fury 58 16 Confirmed 2 13 Kate Winslet 48 14 Confirmed 3 20 Ryan Reynolds 47 20 Confirmed 4 15 Mary Jane 30 15 Confirmed 5 18 Peter Parker 22 18 Confirmed,14.337099
1,"SELECT\n customer_id,\n first_name,\n last_name,\n age,\n reservation_id,\n status\nFROM (\n SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM c.date_of_birth) AS age,\n r.reservation_id,\n r.status\n FROM flight_reservations.customers c\n JOIN flight_reservations.reservations r\n ON c.customer_id = r.customer_id\n WHERE r.status = 'Confirmed'\n AND r.reservation_datetime >= DATE_TRUNC(CURRENT_DATE(), MONTH)\n AND r.reservation_datetime < DATE_ADD(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)\n AND c.date_of_birth < DATE_SUB(CURRENT_DATE(), INTERVAL 18 YEAR)\n) AS t\nORDER BY age DESC;",customer_id first_name last_name age reservation_id status 0 17 Olivia Newton 75 17 Confirmed 1 16 Nick Fury 58 16 Confirmed 2 13 Kate Winslet 48 14 Confirmed 3 20 Ryan Reynolds 47 20 Confirmed 4 15 Mary Jane 30 15 Confirmed 5 18 Peter Parker 22 18 Confirmed,14.847290
2,"SELECT\n customer_id,\n first_name,\n last_name,\n age,\n reservation_id,\n status\nFROM (\n SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM c.date_of_birth) AS age,\n r.reservation_id,\n r.status\n FROM flight_reservations.customers c\n JOIN flight_reservations.reservations r\n ON c.customer_id = r.customer_id\n WHERE r.status = 'Confirmed'\n AND r.reservation_datetime >= DATE_TRUNC(CURRENT_DATE(), MONTH)\n AND r.reservation_datetime < DATE_ADD(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)\n AND c.date_of_birth < DATE_SUB(CURRENT_DATE(), INTERVAL 18 YEAR)\n) AS t\nORDER BY age DESC\nLIMIT 1000;",customer_id first_name last_name age reservation_id status 0 17 Olivia Newton 75 17 Confirmed 1 16 Nick Fury 58 16 Confirmed 2 13 Kate Winslet 48 14 Confirmed 3 20 Ryan Reynolds 47 20 Confirmed 4 15 Mary Jane 30 15 Confirmed 5 18 Peter Parker 22 18 Confirmed,15.433134
3,"SELECT\n c.customer_id,\n c.first_name,\n c.last_name,\n DATE_DIFF(CURRENT_DATE(), c.date_of_birth, YEAR) AS age,\n r.reservation_id,\n r.status\nFROM flight_reservations.customers c\nJOIN flight_reservations.reservations r\nON c.customer_id = r.customer_id\nWHERE r.status = 'Confirmed'\nAND r.reservation_datetime >= DATE_TRUNC(CURRENT_DATE(), MONTH)\nAND r.reservation_datetime < DATE_ADD(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)\nAND c.date_of_birth < DATE_SUB(CURRENT_DATE(), INTERVAL 18 YEAR)\nORDER BY age DESC;",customer_id first_name last_name age reservation_id status 0 17 Olivia Newton 75 17 Confirmed 1 16 Nick Fury 58 16 Confirmed 2 13 Kate Winslet 48 14 Confirmed 3 20 Ryan Reynolds 47 20 Confirmed 4 15 Mary Jane 30 15 Confirmed 5 18 Peter Parker 22 18 Confirmed,20.638619


In [36]:
result_df = sql_output.loc[0, 'Result']
result_df

,customer_id,first_name,last_name,age,reservation_id,status
0,17,Olivia,Newton,75,17,Confirmed
1,16,Nick,Fury,58,16,Confirmed
2,13,Kate,Winslet,48,14,Confirmed
3,20,Ryan,Reynolds,47,20,Confirmed
4,15,Mary,Jane,30,15,Confirmed
5,18,Peter,Parker,22,18,Confirmed
